<a href="https://colab.research.google.com/github/FatherOfLove/256LSA_project/blob/master/models/model1_option_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import os
import pandas as pd
import numpy as np
from google.colab import drive
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization, LSTM, Bidirectional, Input, Concatenate
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
drive.mount('/gdrive')

In [ ]:
option = pd.read_csv('/gdrive/My Drive/Data set/Option/underlying/put/final_put.csv')

In [ ]:
option.head()

In [ ]:
option.info()

In [ ]:
option = option.drop(columns=['Unnamed: 0','UnderlyingSymbol','Exchange','OptionSymbol','Blank','Expiration',
                     'DataDate','Last','Volume','OpenInterest','IV','Delta','Gamma','Theta','Vega','Alias',
                     'sigma_5','sigma_10','sigma_21','sigma_30'])

In [ ]:
option = option.drop(columns=['black_scholes_pred_21'])

In [ ]:
option.info()

In [ ]:
call_df = option

In [ ]:
call_df = call_df.dropna()

In [ ]:
call_df

In [ ]:
call_df.info()

In [ ]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['Bid', 'Ask'], axis=1).values,
                                                                        ((call_df.Bid + call_df.Ask) / 2).values,
                                                                        test_size=0.2, random_state=42)

In [ ]:
call_X_train.shape

In [ ]:
N_TIMESTEPS = 21
call_X_train = [call_X_train[:, -N_TIMESTEPS:].reshape(call_X_train.shape[0], N_TIMESTEPS, 1), call_X_train[:, :4]]
call_X_test = [call_X_test[:, -N_TIMESTEPS:].reshape(call_X_test.shape[0], N_TIMESTEPS, 1), call_X_test[:, :4]]

In [ ]:
layers = 4
n_timesteps = 60
features = 4
n_batch = 1024

In [ ]:
def LSTM_model():
    close_history = Input((N_TIMESTEPS, 1))
    input2 = Input((features,))
    
    lstm = Sequential()
    lstm.add(LSTM(units=8, input_shape=(N_TIMESTEPS, 1), return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=False))
    input1 = lstm(close_history)
    
    connect = Concatenate()([input1, input2])
    
    for _ in range(layers - 1):
        connect = Dense(100)(connect)
        connect = BatchNormalization()(connect)
        connect = LeakyReLU()(connect)
    
    predict = Dense(1, activation='relu')(connect)

    return Model(inputs=[close_history, input2], outputs=predict)

In [ ]:
call_model = LSTM_model()

In [ ]:
call_model.summary()

In [ ]:
call_model.compile(optimizer=Adam(lr=1e-4), loss='mse')
history = call_model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=1000, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1
                    )

In [ ]:
call_model.evaluate(call_X_test, call_y_test, batch_size=512)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
tensorboard --logdir /content/logs/

In [ ]:
call_model.save('/gdrive/My Drive/Data set/Option/lstm_put_2.h5')

In [ ]:
call_y_pred2 = call_model.predict(call_X_test)
'test set mse', np.mean(np.square(call_y_test - np.reshape(call_y_pred2, call_y_pred2.shape[0])))

In [ ]:
call_y_test
for i in range(10):
  print("y: "+ str(call_y_test[i]),", pred: "+ str(call_y_pred2[i]))

In [ ]:
call_y_test

In [ ]:
!tensorboard dev upload \
  --logdir /content/logs/ \
  --name " LSTM2_all_call" \
  --description "n_units = 400,layers = 4,n_batch = 8196,n_epochs = 400,learning_rate=1e-4" \
  --one_shot